In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
import fxcmpy
import time
from datetime import datetime
import pickle
from talib import abstract

/home/mik/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [ ]:
api = fxcmpy.fxcmpy(access_token = "540857ea0f2c6bf6d5bd063300c5f0107fe37ad9")

|WARNING|2023-04-14 15:47:40,273|Default account set to 1347451, to change use set_default_account().


In [ ]:
col = ["tradeId", "amountK", "currency", "grossPL", "isBuy"]

In [ ]:
class MLTrader():
    
    def __init__(self, instrument, bar_length, model, units):
        self.instrument = instrument
        self.bar_length = pd.to_timedelta(bar_length) 
        self.tick_data = None
        self.raw_data = None
        self.data = None 
        self.ticks = 0
        self.last_bar = None  
        self.units = units
        self.feature_columns = ['lag1', 'lag2', 'lag3', 'lag4', 'lag5', 'Ask_Dir', 'SMA', 'SMA_Dir', 'RSI', 'RSI_Dir', 'fastk', 'fastd', 'fastk_Dir', 'fastd_Dir']
        self.position = 0
        self.isBuy = False
        
        #*****************add strategy-specific attributes here******************
        self.lags = 5
        self.model = model
        #************************************************************************        
    
    def get_most_recent(self, period = "m5", number = 10000):
        while True:  
            time.sleep(5)
            df = api.get_candles(self.instrument, number = number, period = period, columns = ["bidclose", "askclose"])
            df[self.instrument] = (df.bidclose + df.askclose) / 2
            df = df[self.instrument].to_frame()
            df = df.resample(self.bar_length, label = "right").last().dropna().iloc[:-1]
            self.raw_data = df.copy()
            self.last_bar = self.raw_data.index[-1]
            if pd.to_datetime(datetime.utcnow()) - self.last_bar < self.bar_length:
                break
    
    def get_tick_data(self, data, dataframe):
        
        self.ticks += 1
        print(self.ticks, end = " ")
        
        recent_tick = pd.to_datetime(data["Updated"], unit = "ms")
        
        # if a time longer than the bar_lenght has elapsed between last full bar and the most recent tick
        if recent_tick - self.last_bar > self.bar_length:
            self.tick_data = dataframe.loc[self.last_bar:, ["Bid", "Ask"]]
            self.tick_data[self.instrument] = (self.tick_data.Ask + self.tick_data.Bid)/2
            self.tick_data = self.tick_data[self.instrument].to_frame()
            self.resample_and_join()
            self.define_strategy() 
            self.execute_trades()
            
            
            
    def resample_and_join(self):
        self.raw_data = self.raw_data.append(self.tick_data.resample(self.bar_length, 
                                                             label="right").last().ffill().iloc[:-1])
        self.last_bar = self.raw_data.index[-1]  
        
    def define_strategy(self): # "strategy-specific"
        df = self.raw_data.copy()
        
        #******************** define your strategy here ************************
        df = df.append(self.tick_data.iloc[-1]) # append latest tick (== open price of current bar)
        df["returns"] = np.log(df[self.instrument] / df[self.instrument].shift())
        cols = []
        for lag in range(1, self.lags + 1):
            col = "lag{}".format(lag)
            df[col] = df.returns.shift(lag)
            cols.append(col)
            
            
        df['Ask_Dir'] = np.where(df[self.instrument].shift(-1) > df[self.instrument], 1, 0)
        cols.append('Ask_Dir')

        df['SMA'] = abstract.SMA(df[self.instrument], timeperiod=12)
        cols.append('SMA')
        df['SMA_Dir'] = np.where(df['SMA'].shift(-1) > df.SMA, 1, 0)
        cols.append('SMA_Dir')

        df['RSI'] = abstract.RSI(df[self.instrument], timeperiod=12)
        cols.append('RSI')
        df['RSI_Dir'] = np.where(df['RSI'].shift(-1) > df.RSI, 1, 0)
        cols.append('RSI_Dir')
        
        df['fastk'], df['fastd'] = abstract.STOCHRSI(df[self.instrument], timeperiod=14, fastk_period=5, fastd_period=3, fastd_matype=0)
        cols.append('fastk')
        cols.append('fastd')
        df['fastk_Dir'] = np.where(df['fastk'].shift(-1) > df.fastk, 1, 0)
        cols.append('fastk_Dir')
        df['fastd_Dir'] = np.where(df['fastd'].shift(-1) > df.fastd, 1, 0)    
        cols.append('fastd_Dir')
            
            
            
        df.dropna(inplace = True)
        df["position"] = self.model.predict(df[self.feature_columns])
        #***********************************************************************
                
        self.data = df.copy()
    
    def execute_trades(self):
        
        if self.position == 1:
            if (self.data["position"].iloc[-1] == -1 and self.isBuy) or (self.data["position"].iloc[1] == 1 and not self.isBuy):
                order = api.close_all_for_symbol(self.instrument)
                self.report_trade(order, "Close position")
                self.position = 0
                self.isBuy = False
            
            
        if self.position == 0 and self.data["position"].iloc[-1] == -1: 
            print('new prediction is short but current position is long')
            # close existing long position, and create new short
            api.close_all_for_symbol(symbol= self.instrument)
            order = api.create_market_sell_order(self.instrument, self.units)
            self.report_trade(order, "GOING SHORT")
            self.position = 1
            self.isBuy = False
                 
        if self.position == 0 and self.data["position"].iloc[-1] == 1: 
            print('new prediction is long but current position is short') 
            # close existing short position, and create new long
            api.close_all_for_symbol(symbol= self.instrument)
            order = api.create_market_buy_order(self.instrument, self.units)
            self.report_trade(order, "GOING LONG")
            self.position = 1
            self.isBuy = True
            
        if self.data["position"].iloc[-1] == 0: 
            if self.position == -1:
                order = api.close_all_for_symbol(symbol= self.instrument)
                self.report_trade(order, "GOING NEUTRAL") 
            elif self.position == 1:
                order = api.close_all_for_symbol(symbol= self.instrument)
                self.report_trade(order, "GOING NEUTRAL")  
            self.position = 0


    def report_trade(self, order, going):
        time = order.get_time()
        units = api.get_open_positions().amountK.iloc[-1]
        price = api.get_open_positions().open.iloc[-1]
        unreal_pl = api.get_open_positions().grossPL.sum()
        print("\n" + 100* "-")
        print("{} | {}".format(time, going))
        print("{} | units = {} | price = {} | Unreal. P&L = {}".format(time, units, price, unreal_pl))
        print(100 * "-" + "\n")
        

In [ ]:
lm = pickle.load(open("logreg.pkl", "rb"))


In [ ]:
# Create ML Trader object
trader = MLTrader("EUR/USD", "5min", model = lm, units = 100)

In [ ]:
# Start online trading

trader.get_most_recent()
api.subscribe_market_data(trader.instrument, (trader.get_tick_data, ))

1 2 3 4 

In [ ]:
# Close online trading
#api.unsubscribe_market_data(trader.instrument)
#if len(api.get_open_positions()) != 0: # if we have final open position(s) (netting and hedging)
#    api.close_all_for_symbol(trader.instrument)
#    print(2*"\n" + "{} | GOING NEUTRAL".format(str(datetime.utcnow())) + "\n")
#    time.sleep(20)
#    print(api.get_closed_positions_summary()[col])
#    trader.position = 0


5 